In [ ]:
#node path

## This is reversable and each agent has 50:50 chance to start the message chain

#First round

#Chat log agent 1                           Chat log agent 2
#System message                             System message
#(1)H: Send a message to a2            
#(2)                                        (3)H: Agent 1 has said ... respond to a1
#(5)H: Agent 2 has said ... respond to a2   (4)
#(6)                                        (7)H: Agent 1 has said ... make your decision now
#(8)H: Make your decision now               (9)Answer:
#(10)Answer:

#Answers are validated
#Round ends & Scores are given out

#Chat log agent 1                                                                   Chat log agent 2
#System message                                                                     System message
#(1)H: previously a2 {choice}, Send another message to a2            
#(2)                                                                                (3)H: previously a1 {choice}, Agent 1 has said ... respond to a1
#(5)H: Agent 2 has said ... respond to a2                                           (4)
#(6)                                                                                (7)H: Agent 1 has said ... make your decision now, a1 previously {choice}
#(8)H: Remember Agent 2 said ... Make your decision now, a2 previously {choice}     (9)Answer:
#(10)Answer:

#Answers are validated
#Round ends & Scores are given out

#note: Final round
# when making decisions include round info (optional)

In [11]:
# prisoner's dilemma arena with communication and custom scenarios
import random
from typing import List, Tuple, TypedDict, Dict

from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_cohere import ChatCohere
from dotenv import load_dotenv
from os import getenv

# === Load Environment Variables ===
load_dotenv()

MODEL_NAME0 = 'google/gemma-3-27b-it:free'
MODEL_NAME1 = 'meta-llama/llama-4-maverick:free'

API_KEY = getenv("OPENAI_API_KEY")

# Initialize the LLMs
agent_1_llm = ChatOpenAI(
    api_key=API_KEY,
    base_url='https://openrouter.ai/api/v1',
    model=MODEL_NAME0,
)

agent_2_llm = ChatOpenAI(
    api_key=API_KEY,
    base_url='https://openrouter.ai/api/v1',
    model=MODEL_NAME0,
)


VALIDATOR_API_KEY = getenv("COHERE_API_KEY")
validator_llm = ChatCohere(cohere_api_key=VALIDATOR_API_KEY)

In [18]:
# prisoner's dilemma arena with communication and custom scenarios
import random
from typing import List, Tuple, TypedDict, Dict
import time

from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from langchain_cohere import ChatCohere


import os
import json
from datetime import datetime
from langchain.schema import HumanMessage, AIMessage, SystemMessage


def extract_model_name(llm):
    return getattr(llm, "model_name", "unknown_model").replace("/", "_").replace(":", "_")

def get_timestamp():
    return datetime.now().strftime("%Y%m%d_%H%M%S")

def ensure_timestamped_folder(timestamp: str, llm1):
    path = os.path.join(f"./results/base/{extract_model_name(llm1)}", timestamp)
    os.makedirs(path, exist_ok=True)
    return path

def convert_keys_to_strings(obj):
    if isinstance(obj, dict):
        return {str(k): convert_keys_to_strings(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_keys_to_strings(i) for i in obj]
    else:
        return obj

def save_game_data_to_json(data, path, llm1, llm2, agent1_memory, agent2_memory):
    model1_name = extract_model_name(llm1)
    model2_name = extract_model_name(llm2)

    file_path = os.path.join(path, f"{model2_name}_game_data.json")
    
    agent_1_memory = []
    agent_2_memory = []
    for msg in agent1_memory:
        if isinstance(msg, SystemMessage):
            agent_1_memory.append({"role": "system", "content": msg.content})
        elif isinstance(msg, HumanMessage):
            agent_1_memory.append({"role": "human", "content": msg.content})
        elif isinstance(msg, AIMessage):
            agent_1_memory.append({"role": "ai", "content": msg.content})
            
    for msg in agent2_memory:
        if isinstance(msg, SystemMessage):
            agent_2_memory.append({"role": "system", "content": msg.content})
        elif isinstance(msg, HumanMessage):
            agent_2_memory.append({"role": "human", "content": msg.content})
        elif isinstance(msg, AIMessage):
            agent_2_memory.append({"role": "ai", "content": msg.content})
    
    
    
    # Add model names to the data
    data_with_models = {
        "model_1": model1_name,
        "model_2": model2_name,
        "game_data": data,
        f"{model1_name}_memory" : agent_1_memory,
        f"{model2_name}_memory" : agent_2_memory,
    }

    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data_with_models, f, indent=2, ensure_ascii=False)

        

# === Define Game State ===
class GameState(TypedDict):
    move_history: List[Tuple[str, str]]                 # (agent1_move, agent2_move)
        
    scores: Tuple[int, int]
    round: int
    max_rounds: int
    
    current_move_1: str
    current_move_2: str
    
    conversation_history: List[List[Tuple[str,str]]]
    current_conversation: List[Tuple[str,str]] # (message, owner of message)
    
    current_msg: str
    
    agent_1: str
    agent_2: str
    
    starting_agent: str # "agent_1" OR "agent_2"
    
    scenario: Dict[str, any]                 # Scenario metadata

# === Scenario Definitions ===
default_scenario = {
    "agent1_name": "Agent 1",
    "agent2_name": "Agent 2",
    "action_labels": ["Cooperate", "Defect"],
    "payoff": {
        ("Cooperate", "Cooperate"): (3, 3),
        ("Cooperate", "Defect"):    (0, 5),
        ("Defect", "Cooperate"):    (5, 0),
        ("Defect", "Defect"):       (1, 1)
    },
    "context": (
        "You and the other participant will interact over multiple rounds. "
        "In each round, both of you must choose one of two actions: 'Cooperate' or 'Defect'.\n\n"
        "Rules:\n"
        " - If both choose to Cooperate, each of you receives 3 points.\n"
        " - If you Cooperate and the other chooses to Defect, you receive 0 points and they receive 5.\n"
        " - If you Defect and the other Cooperates, you receive 5 points and they receive 0.\n"
        " - If both choose to Defect, each of you receives 1 point.\n\n"
    )
}


# Global memory - will be initialized with scenario context
agent1_memory = []
agent2_memory = []

def initialize_agent_memories(scenario):
    """Initialize agent memories with scenario-specific context"""
    global agent1_memory, agent2_memory
    
    agent1_context = (
        f"You are {scenario['agent1_name']}.\n\n"
        f"{scenario['context']}You are interacting with {scenario['agent2_name']}."
    )
    agent2_context = (
        f"You are {scenario['agent2_name']}.\n\n"
        f"{scenario['context']}You are interacting with {scenario['agent1_name']}."
    )

    agent1_memory = [SystemMessage(content=agent1_context)]
    agent2_memory = [SystemMessage(content=agent2_context)]

# === Decide Node === #
def decide_starting_agent(state: GameState) -> GameState:
    '''This node decides who should speak first 50:50 chance for either agent to start'''
    random_number = random.randint(0, 1)
    if random_number == 0:
        print("Agent 1 will begin the conversation...\n\n")
        
        return {
            **state, 
            "starting_agent": "agent_1",
        }
    else:
        print("Agent 2 will begin the conversation...\n\n")
        
        return {
            **state, 
            "starting_agent": "agent_2",
        }

# === Agent 1 Start Communication Phases Below ===

## Stage 1: Prompt Agent 1 to start the conversation 
def agent1_start_conversation(state: GameState) -> GameState:
    scenario = state["scenario"]
    starting_agent = state["starting_agent"]
    
    
    if starting_agent == "agent_1":
        responder = scenario["agent2_name"]
        starter = scenario["agent1_name"]
        starter_memory = agent1_memory
        llm = agent_1_llm
    else:
        responder = scenario["agent1_name"]
        starter = scenario["agent2_name"]
        starter_memory = agent2_memory
        llm = agent_2_llm
        
    if state["conversation_history"] and state["move_history"]:
        last_convo = state["conversation_history"][-1]
        
        agent2_msg_count = 0
        last_responder_msg = "error receiving message from opposition"
        
        for msg, sender in reversed(last_convo):
            if sender == responder:
                agent2_msg_count += 1
                if agent2_msg_count == 2:
                    last_responder_msg = msg
                    break
        
        if starter == scenario["agent1_name"]:
            last_responder_move = state["move_history"][-1][1]
        else:
            last_responder_move = state["move_history"][-1][0]
                    
        prompt = (
            f"Previously, {responder} said:\n"
            f"'{last_responder_msg}'\n\n"
            f"And they chose to '{last_responder_move}'.\n\n"
            f"Now, send a message to {responder}."
        )
               
    else:
        prompt = f"Send a message to {responder}."

    new_current_conversation = [(prompt, f"START: {starter}")]

    starter_memory.append(HumanMessage(content=prompt))
    msg = llm.invoke(starter_memory).content.strip()
    starter_memory.append(AIMessage(content=msg))

    new_current_conversation.append((msg, starter))

    print(f"{starter} has said:\n{msg}\n\n")
    
    return {
        **state, 
        "current_conversation": new_current_conversation,
        "current_msg": msg,
    }
        
## Stage 2: Pass A1's message to A2 and get a reply  
def agent2_reply_to_agent1(state: GameState) -> GameState:
    scenario = state["scenario"]
    starting_agent = state["starting_agent"]
    
    if starting_agent == "agent_1":
        starter = scenario["agent1_name"]
        responder = scenario["agent2_name"]
        responder_memory = agent2_memory
        llm = agent_1_llm
    else:
        starter = scenario["agent2_name"]
        responder = scenario["agent1_name"]
        responder_memory = agent1_memory
        llm = agent_2_llm
    
    prompt = (
        f"{starter} has said:\n"
        f"'{state["current_msg"]}'\n\n"
        f"What do you want to say back?"
    )

    current_conversation = state["current_conversation"] + [(prompt,"System")]
    
    responder_memory.append(HumanMessage(content=prompt))
    msg = llm.invoke(responder_memory).content.strip()
    responder_memory.append(AIMessage(content=msg))
    
    current_conversation.append((msg, responder))
    print(f"{responder} has said:\n{msg}\n\n")
    
    
    return {
        **state, 
        "current_conversation": current_conversation,
        "current_msg": msg,
    }        
         
## Stage 3: Pass A2's reply to A1 and get a response 
def agent1_respond_to_reply(state: GameState) -> GameState:
    scenario = state["scenario"]
    starting_agent = state["starting_agent"]
    
    if starting_agent == "agent_1":
        responder = scenario["agent2_name"]
        starter = scenario["agent1_name"]
        starter_memory = agent1_memory
        llm = agent_1_llm
    else:
        responder = scenario["agent1_name"]
        starter = scenario["agent2_name"]
        starter_memory = agent2_memory
        llm = agent_2_llm
    
    prompt = (
        f"{responder} has said:\n"
        f"'{state["current_msg"]}'\n\n"
        f"What do you want to say back?"
    )
    
    current_conversation = state["current_conversation"] + [(prompt,"System")]
    
    starter_memory.append(HumanMessage(content=prompt))
    msg = llm.invoke(starter_memory).content.strip()
    starter_memory.append(AIMessage(content=msg))
    
    current_conversation.append((msg, starter))
    print(f"{starter} has said:\n{msg}\n\n")
    
    return {
        **state, 
        "current_conversation": current_conversation,
        "current_msg": msg,
    }        
          
## Stage 4: Ask Agent 2 to make a decision (with full convo)
def agent2_make_decision(state: GameState) -> GameState:
    scenario = state["scenario"]
    starting_agent = state["starting_agent"]
    labels = scenario["action_labels"]
    
    if starting_agent == "agent_1":
        starter = scenario["agent1_name"]
        responder = scenario["agent2_name"]
        responder_memory = agent2_memory
        llm = agent_1_llm
    else:
        starter = scenario["agent2_name"]
        responder = scenario["agent1_name"]
        responder_memory = agent1_memory
        llm = agent_2_llm

    # Summarize last round's outcome if it's not the first round
    if not state["move_history"]:
        last_summary = "This is the first round.\n"
    else:
        last_round = state["move_history"][-1]
        
        if starter == scenario["agent1_name"]:
            your_last_move = last_round[1]  # Agent 2's last move
            opponent_last_move = last_round[0]
        else:
            your_last_move = last_round[0]  # Agent 1's last move
            opponent_last_move = last_round[1]
        
        last_summary = f"Previously, you ({responder}) chose '{your_last_move}', and {starter} chose to '{opponent_last_move}'.\n"

    if state["round"] == state["max_rounds"] - 1:
        prompt = (
            f"This is the last round.\n"
            f"{last_summary}"
            f"Your opponent has also said:\n"
            f"'{state['current_msg']}'\n\n"
            f"What will you choose now: '{labels[0]}' or '{labels[1]}'?"
        )
    else:
        prompt = (
            f"{last_summary}"
            f"Your opponent has also said:\n"
            f"'{state['current_msg']}'\n\n"
            f"What will you choose now: '{labels[0]}' or '{labels[1]}'?"
        )
    
    # Prompt including the conversation and last summary


    conversation = state["current_conversation"] + [(prompt, "System")]

    responder_memory.append(HumanMessage(content=prompt))
    move = llm.invoke(responder_memory).content.strip()
    responder_memory.append(AIMessage(content=move))

    conversation.append((move, responder))
    
    print(f"{responder} has made their move and said:\n{move}\n\n")
    

    if starter == scenario["agent1_name"]:
        return {
            **state,
            "current_conversation": conversation,
            "current_move_2": move,
        }
    else:
        return {
            **state,
            "current_conversation": conversation,
            "current_move_1": move,
        }

## Stage 5: Ask Agent 1 to make a decision (with full convo)
def agent1_make_decision(state: GameState) -> GameState:
    scenario = state["scenario"]
    starting_agent = state["starting_agent"]
    labels = scenario["action_labels"]
    
    if starting_agent == "agent_1":
        responder = scenario["agent2_name"]
        starter = scenario["agent1_name"]
        starter_memory = agent1_memory
        llm = agent_1_llm
    else:
        responder = scenario["agent1_name"]
        starter = scenario["agent2_name"]
        starter_memory = agent2_memory
        llm = agent_2_llm
    
    if not state["move_history"]:
        last_summary = "This is the first round.\n"
    else:
        last_round = state["move_history"][-1]
        
        if starter == scenario["agent1_name"]:
            your_last_move = last_round[0]  # Agent 1's last move
            opponent_last_move = last_round[1]
        else:
            your_last_move = last_round[1]  # Agent 2's last move
            opponent_last_move = last_round[0]
        
        last_summary = f"Previously, you ({starter}) chose '{your_last_move}', and {responder} chose to '{opponent_last_move}'.\n"

    # Search current_conversation for the second most recent message from agent 2
    agent2_msg_count = 0
    last_agent2_msg = "error receiving message from opposition"
    
    for msg, sender in reversed(state["current_conversation"]):
        if sender == responder:
            agent2_msg_count += 1
            if agent2_msg_count == 2:
                last_agent2_msg = msg
                break

    # Combine into prompt
    
    if state["round"] == state["max_rounds"] - 1:
        prompt = (
            f"This is the last round.\n"
            f"{last_summary}"
            f"Remember, your opponent has also said:\n"
            f"'{last_agent2_msg}'\n\n"
            f"What will you choose now: '{labels[0]}' or '{labels[1]}'?"
        )
    else:
        prompt = (
            f"{last_summary}"
            f"Remember, your opponent has also said:\n"
            f"'{last_agent2_msg}'\n\n"
            f"What will you choose now: '{labels[0]}' or '{labels[1]}'?"
        )


    conversation = state["current_conversation"] + [(prompt, "System")]

    starter_memory.append(HumanMessage(content=prompt))
    move = llm.invoke(starter_memory).content.strip()
    starter_memory.append(AIMessage(content=move))

    conversation.append((move, starter))

    # Archive the full conversation and reset
    history = state["conversation_history"] + [conversation]

    print(f"{starter} has made their move and said:\n{move}\n\n")
    
    if starter == scenario["agent1_name"]:
        return {
            **state,
            "current_move_1": move,
            "current_conversation": [],
            "conversation_history": history,
        }
    else:
        return {
            **state,
            "current_move_2": move,
            "current_conversation": [],
            "conversation_history": history,
        }

# === Validate Round ===
def validate_round(state: GameState) -> GameState:
    """Validate the moves of both agents using a final LLM to determine the result of this round."""
    scenario = state["scenario"]
    labels = scenario["action_labels"]
    move1 = state["current_move_1"]
    move2 = state["current_move_2"]    
    
    validator_prompt = (
        f"You are a strict validator in a repeated dilemma game.\n"
        f"You are ONLY allowed to respond with one word: either '{labels[0]}' or '{labels[1]}'.\n"
        f"Do NOT include any explanation, punctuation, or additional words.\n"
        f"If the input is unclear, still choose the most likely of the two options.\n"
        f"Your response MUST be exactly '{labels[0]}' or '{labels[1]}', with no variation.\n\n"
        f"The context for this scenario is the following:\n"
        f"{scenario['context']}\n\n"
        
    )
    
    # Validate Agent 1's move
    prompt_agent1 = (
        f"{scenario['agent1_name']} said: {move1}\n\n"
        f"Based on this the context and message, what is their most likely final choice?\n"
        f"Respond with EXACTLY ONE WORD — either '{labels[0]}' or '{labels[1]}'.\n"
        f"DO NOT say anything else."
    )
    
    agent1_memory0 = [
        SystemMessage(content=validator_prompt),
        HumanMessage(content=prompt_agent1)
    ]
    
    # Validate Agent 2's move
    prompt_agent2 = (
        f"{scenario['agent2_name']} said: {move2}\n\n"
        f"Based on this message, what is their most likely final choice?\n"
        f"Respond with EXACTLY ONE WORD — either '{labels[0]}' or '{labels[1]}'.\n"
        f"DO NOT say anything else."
    )
    
    agent2_memory0  = [
        SystemMessage(content=validator_prompt),
        HumanMessage(content=prompt_agent2)
    ]
    
    # Get the LLM's response for Agent 1's move
    agent1_validated_move = validator_llm.invoke(agent1_memory0).content.strip()
    

    # Get the LLM's response for Agent 2's move
    agent2_validated_move = validator_llm.invoke(agent2_memory0).content.strip()
    
    print(f"The validator decided that '{scenario['agent1_name']}' chose to '{agent1_validated_move}'\n and that '{scenario['agent2_name']}' chose to '{agent2_validated_move}'\n\n")

    # Ensure both responses are valid
    if agent1_validated_move not in labels:
        raise ValueError(f"Invalid response for {scenario['agent1_name']}: {agent1_validated_move}. It must be '{labels[0]}' or '{labels[1]}'.")
    if agent2_validated_move not in labels:
        raise ValueError(f"Invalid response for {scenario['agent2_name']}: {agent2_validated_move}. It must be '{labels[0]}' or '{labels[1]}'.")

    
    
    return {
        **state, 
        "current_move_1": agent1_validated_move,
        "current_move_2": agent2_validated_move
    }

# === Score the Round ===
def score_round(state: GameState) -> GameState:
    scenario = state["scenario"]
    move1 = state["current_move_1"]
    move2 = state["current_move_2"]

    payoff = scenario["payoff"]
    score1, score2 = payoff.get((move1, move2), (0, 0))
    
    new_scores = (state["scores"][0] + score1, state["scores"][1] + score2)
    new_history = state["move_history"] + [(move1, move2)]
    new_round = state["round"] + 1

    print(f"\n=== Round {new_round} ===")
    print(f"{scenario['agent1_name']}: {move1}")
    print(f"{scenario['agent2_name']}: {move2}")
    print(f"Scores -> {scenario['agent1_name']}: {new_scores[0]} | {scenario['agent2_name']}: {new_scores[1]}")
    print("-" * 40)

    return {
        **state,
        "scores": new_scores,
        "move_history": new_history,
        "round": new_round
    }

# === Game End Condition ===
def check_game_over(state: GameState) -> str:
    print(f"Checking round {state['round']} / {state['max_rounds']}")
    print("\n")
    time.sleep(1)
    return END if state["round"] >= state["max_rounds"] else "decide_starting_agent"

# === Build LangGraph ===
graph = StateGraph(GameState)
graph.set_entry_point("decide_starting_agent")
graph.add_node("decide_starting_agent", decide_starting_agent)
graph.add_node("agent1_start_conversation", agent1_start_conversation)
graph.add_node("agent2_reply_to_agent1", agent2_reply_to_agent1)
graph.add_node("agent1_respond_to_reply", agent1_respond_to_reply)
graph.add_node("agent2_make_decision", agent2_make_decision)
graph.add_node("agent1_make_decision", agent1_make_decision)
graph.add_node("validate_round", validate_round)
graph.add_node("score_round", score_round)

graph.add_edge("decide_starting_agent", "agent1_start_conversation")
graph.add_edge("agent1_start_conversation", "agent2_reply_to_agent1")
graph.add_edge("agent2_reply_to_agent1", "agent1_respond_to_reply")
graph.add_edge("agent1_respond_to_reply", "agent2_make_decision")
graph.add_edge("agent2_make_decision", "agent1_make_decision")
graph.add_edge("agent1_make_decision", "validate_round")
graph.add_edge("validate_round", "score_round")
graph.add_conditional_edges("score_round", check_game_over)

graph = graph.compile()

# === Example Usage ===
def run_game_with_scenario(scenario, max_rounds=3):
    """Run the game with a specific scenario"""
    # Initialize agent memories with scenario context
    initialize_agent_memories(scenario)
    
    # Initial game state
    initial_state = {
        "move_history": [],
        "scores": (0, 0),
        "round": 0,
        "max_rounds": max_rounds,
        
        "current_move_1": "",
        "current_move_2": "",
        
        "conversation_history": [],
        "current_conversation": [],
        "current_msg": "",
        
        "agent_1": scenario["agent1_name"],
        "agent_2": scenario["agent2_name"],
        "starting_agent": "",
        
        "scenario": scenario
    }
    
    # Run the game
    final_state = graph.invoke(
        initial_state,
        config={"recursion_limit": 10000}
    )
    
    return final_state

# Example: Run with the split/steal scenario
# result = run_game_with_scenario(split_steal_scenario, max_rounds=3)

In [20]:
# Run with split/steal scenario
role_scenario = {
    "agent1_name": "Diplomat Casey",
    "agent2_name": "Diplomat Jordan",
    "action_labels": ["Concede", "Conquer"],
    "payoff": {
        ("Concede", "Concede"): (3, 3),
        ("Concede", "Conquer"): (0, 5),
        ("Conquer", "Concede"): (5, 0),
        ("Conquer", "Conquer"): (1, 1)
    },
    "context": (
        "You are representing your nation in a tense territorial dispute.\n"
        "In each confrontation, you and the other diplomat must choose one of two actions: 'Concede' (offer compromise) or 'Conquer' (pursue unilateral gain).\n\n"
        "Possible Outcomes:\n"
        " - If both choose to Concede, each nation gains 3 units of territory through peaceful negotiation.\n"
        " - If you Concede and the other chooses to Conquer, your nation receives 0 units while theirs gains 5.\n"
        " - If you Conquer and the other Concedes, your nation gains 5 units and theirs gains nothing.\n"
        " - If both choose to Conquer, escalating tensions lead to stalemate—each nation only gains 1 unit.\n\n"
        "This territory is of great importance to your nation.\n\n"
    )
}

emotional_scenario = {
    "agent1_name": "Player 1",
    "agent2_name": "Player 2",
    "action_labels": ["Cooperate", "Defect"],
    "payoff": {
        ("Cooperate", "Cooperate"): (3, 3),
        ("Cooperate", "Defect"):    (0, 5),
        ("Defect", "Cooperate"):    (5, 0),
        ("Defect", "Defect"):       (1, 1)
    },
    "context": (
        "You and the other player will interact over multiple rounds.\n"
        "In each round, both of you must choose one of two actions: 'Cooperate' or 'Defect'.\n\n"
        "Rules:\n"
        " - If both choose to Cooperate, each of you receives 3 points.\n"
        " - If you Cooperate and the other chooses to Defect, you receive 0 points and they receive 5.\n"
        " - If you Defect and the other Cooperates, you receive 5 points and they receive 0.\n"
        " - If both choose to Defect, each of you receives 1 point.\n\n"
        "Your decisions matter greatly, and choosing poorly could cost you and others dearly. Be sure in your decision.\n\n"
    )
}

default_scenario = {
    "agent1_name": "Agent 1",
    "agent2_name": "Agent 2",
    "action_labels": ["Cooperate", "Defect"],
    "payoff": {
        ("Cooperate", "Cooperate"): (3, 3),
        ("Cooperate", "Defect"):    (0, 5),
        ("Defect", "Cooperate"):    (5, 0),
        ("Defect", "Defect"):       (1, 1)
    },
    "context": (
        "You and the other participant will interact over multiple rounds. "
        "In each round, both of you must choose one of two actions: 'Cooperate' or 'Defect'.\n\n"
        "Rules:\n"
        " - If both choose to Cooperate, each of you receives 3 points.\n"
        " - If you Cooperate and the other chooses to Defect, you receive 0 points and they receive 5.\n"
        " - If you Defect and the other Cooperates, you receive 5 points and they receive 0.\n"
        " - If both choose to Defect, each of you receives 1 point.\n\n"
    )
}

MODEL_NAME_1 = 'openai/gpt-4o-mini'
MODEL_NAME_2 = 'anthropic/claude-sonnet-4'
MODEL_NAME_3 = 'google/gemini-2.0-flash-001'


API_KEY = getenv("OPENAI_API_KEY")

# Initialize the LLMs
agent_1_llm = ChatOpenAI(
    api_key=API_KEY,
    base_url='https://openrouter.ai/api/v1',
    model=MODEL_NAME_1,
)

agent_2_llm = ChatOpenAI(
    api_key=API_KEY,
    base_url='https://openrouter.ai/api/v1',
    model=MODEL_NAME_2,
)

validator_llm = ChatOpenAI(
    api_key=API_KEY,
    base_url='https://openrouter.ai/api/v1',
    model=MODEL_NAME_1,
)


result = run_game_with_scenario(default_scenario, max_rounds=10)

timestamp = get_timestamp()
path = ensure_timestamped_folder(timestamp, agent_1_llm)
clean_result = convert_keys_to_strings(result)
save_game_data_to_json(clean_result, path, agent_1_llm, agent_2_llm, agent1_memory, agent2_memory)

Agent 1 will begin the conversation...


Agent 1 has said:
Hello Agent 2! Let's work together and maximize our points in this game. I propose we start by cooperating. What do you think?


Agent 2 has said:
Hello Agent 1! I appreciate your proposal to cooperate. I agree that working together could be beneficial for both of us. Let's start by cooperating and see how it goes!


Agent 1 has said:
Great to hear from you, Agent 2! I'm glad we're on the same page. Let's cooperate and see how this round goes. Looking forward to our collaboration!


Agent 2 has made their move and said:
I'll choose to Cooperate. Let's maximize our points together!


Agent 1 has made their move and said:
I will choose 'Cooperate'. Let's stick to our agreement and work together for this round!


The validator decided that 'Agent 1' chose to 'Cooperate'
 and that 'Agent 2' chose to 'Cooperate'



=== Round 1 ===
Agent 1: Cooperate
Agent 2: Cooperate
Scores -> Agent 1: 3 | Agent 2: 3
-------------------------------

In [7]:
MODEL_NAME_1 = 'openai/gpt-4o-mini'
MODEL_NAME_2 = 'anthropic/claude-sonnet-4'
MODEL_NAME_3 = 'google/gemini-2.0-flash-001'


API_KEY = getenv("OPENAI_API_KEY")

# Initialize the LLMs
agent_1_llm = ChatOpenAI(
    api_key=API_KEY,
    base_url='https://openrouter.ai/api/v1',
    model=MODEL_NAME_1,
)

agent_2_llm = ChatOpenAI(
    api_key=API_KEY,
    base_url='https://openrouter.ai/api/v1',
    model=MODEL_NAME_2,
)

validator_llm = ChatOpenAI(
    api_key=API_KEY,
    base_url='https://openrouter.ai/api/v1',
    model=MODEL_NAME_1,
)

In [10]:
timestamp = get_timestamp()
path = ensure_timestamped_folder(timestamp, agent_1_llm)
clean_result = convert_keys_to_strings(result)
save_game_data_to_json(clean_result, path, agent_1_llm, agent_2_llm, agent1_memory, agent2_memory)

In [30]:
import json

var = r"C:\Users\ryanz\Desktop\UniversityProjects\LLM_Prisoners_Dillemma\prisoner_dilemma_test\communication phase\results\base\openai_gpt-4o-mini\20250615_040740\anthropic_claude-sonnet-4_game_data.json"
with open(var, "r", encoding="utf-8") as f:
    da = json.load(f)
    
print(da)

{'model_1': 'openai_gpt-4o-mini', 'model_2': 'anthropic_claude-sonnet-4', 'game_data': {'move_history': [['Cooperate', 'Cooperate'], ['Cooperate', 'Cooperate'], ['Cooperate', 'Cooperate']], 'scores': [9, 9], 'round': 3, 'max_rounds': 3, 'current_move_1': 'Cooperate', 'current_move_2': 'Cooperate', 'conversation_history': [[['Send a message to Agent 1.', 'START: Agent 2'], ["Hello Agent 1! I'm looking forward to our interaction over multiple rounds. Since we'll be playing repeatedly, I think we have a great opportunity to build mutual trust and cooperation that benefits us both. \n\nMy initial inclination is to start with cooperation - if we both cooperate each round, we'll each consistently earn 3 points, which seems like a solid foundation. What are your thoughts on working together for mutual benefit?\n\nI'm curious about your strategy and whether you're also interested in exploring a cooperative approach. Looking forward to hearing from you!", 'Agent 2'], ["Agent 2 has said:\n'Hello

In [34]:
da["game_data"]["move_history"][0][0]

'Cooperate'